In [ ]:
'''
Flower classification using PyTorch
dataset: https://www.kaggle.com/alxmamaev/flowers-recognition
'''

'\nFlower classification using PyTorch\ndataset: https://www.kaggle.com/alxmamaev/flowers-recognition\n'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [ ]:
!ls

 flowers				     resnet50_FC_features_512.csv
 kaggle.json				    'resnet50_TL_model_94(1)%acc.pth'
 my.png					     resnet50_TL_model_94%acc.pth
 resnet50_FC_512_features_with_labels1.csv


In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
%pwd

/content/gdrive/My Drive/Kaggle


'/content/gdrive/My Drive/Kaggle'

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /content/gdrive/My Drive/kaggle. Or use the environment method.


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [ ]:
DIR_PATH = '/content/gdrive/MyDrive/Kaggle/flowers/flowers'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pytorch imports

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

transformations = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [ ]:
# hyperparamters

learning_rate = 0.001
batch_size = 8
num_epochs = 50
num_classes = 5

# device
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(device)

cpu


In [ ]:
total_dataset = torchvision.datasets.ImageFolder(DIR_PATH,transform=transformations['train'])

len(total_dataset),total_dataset[0][0].shape,total_dataset.class_to_idx

(4323,
 torch.Size([3, 224, 224]),
 {'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4})

In [ ]:
# splitting into train and validation sets

SPLIT_SIZE = 0.8
tot_len = len(total_dataset)

train_size = int(SPLIT_SIZE * tot_len)
val_size = tot_len - train_size

print(f'Training set size = {train_size} \nValidation set size = {val_size}')

train_dataset, val_dataset =  torch.utils.data.random_split(total_dataset,[train_size,val_size])

prediction_dataset=total_dataset[0][0]


print(prediction_dataset)

len(train_dataset),len(val_dataset)

Training set size = 3458 
Validation set size = 865
tensor([[[ 0.2111,  0.2796,  0.3652,  ...,  0.5193,  0.5193,  0.4679],
         [ 0.1939,  0.2624,  0.3481,  ...,  0.5193,  0.5193,  0.4508],
         [ 0.1768,  0.2624,  0.3481,  ...,  0.5022,  0.5022,  0.4166],
         ...,
         [-1.3473, -1.3644, -1.3644,  ...,  0.1083,  0.1083,  0.0912],
         [-1.3644, -1.3644, -1.3644,  ...,  0.1254,  0.1083,  0.1083],
         [-1.3644, -1.3644, -1.3644,  ...,  0.1426,  0.1426,  0.1426]],

        [[ 0.3452,  0.4153,  0.5028,  ...,  0.6604,  0.6604,  0.6078],
         [ 0.3277,  0.3978,  0.4853,  ...,  0.6604,  0.6604,  0.5903],
         [ 0.3102,  0.3978,  0.4853,  ...,  0.6429,  0.6429,  0.5553],
         ...,
         [-1.1954, -1.2129, -1.2129,  ...,  0.1702,  0.1702,  0.1527],
         [-1.2129, -1.2129, -1.2129,  ...,  0.1877,  0.1702,  0.1702],
         [-1.2129, -1.2129, -1.2129,  ...,  0.2052,  0.2052,  0.2052]],

        [[ 0.5311,  0.6356,  0.7228,  ...,  0.8448,  0.8797,  0.

(3458, 865)

In [ ]:
prediction_dataset.size()

torch.Size([3, 224, 224])

In [ ]:
# dataloaders
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=4)

val_loader = DataLoader(dataset=val_dataset,
                       batch_size=1,
                       shuffle=True,
                       num_workers=4)
prediction_loader=DataLoader(dataset=prediction_dataset,batch_size=1,shuffle=False,num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# testing dataloading 

examples = iter(train_loader)
samples,labels = examples.next()
print(samples.shape,labels.shape) # batch_size=8
len(train_loader),len(val_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([8, 3, 224, 224]) torch.Size([8])


(433, 865)

In [ ]:
# custom CNN model class

class ConvNet(nn.Module):
    def __init__(self,model,num_classes):
        super(ConvNet,self).__init__()
        self.base_model = nn.Sequential(*list(model.children())[:-1]) # model excluding last FC layer
        self.linear1 = nn.Linear(in_features=2048,out_features=512)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=512,out_features=num_classes)
    
    def forward(self,x):
        x = self.base_model(x)
        x = torch.flatten(x,1)
        lin = self.linear1(x)
        x = self.relu(lin)
        out = self.linear2(x)
        return lin, out

In [ ]:
model = torchvision.models.resnet50(pretrained=True) # base model

model = ConvNet(model,num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

In [ ]:
print(model)

ConvNet(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(6

In [ ]:
# training loop

n_iters = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    for ii,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        _,outputs = model(images)
        loss = criterion(outputs,labels)
        
        # free_gpu_cache()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (ii+1)%108 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{ii+1}/{n_iters}], Loss = {loss.item():.6f}')
            
    print('----------------------------------------')
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [1/50], Step [108/433], Loss = 0.998556


KeyboardInterrupt: ignored

In [ ]:
# evaluating model and getting features of every image

def eval_model_extract_features(features,true_labels,model,dataloader,phase):

    with torch.no_grad():
        # for entire dataset
        n_correct = 0
        n_samples = 0

        model.eval()

        for images,labels in dataloader:

            images = images.to(device)
            labels = labels.to(device)

            true_labels.append(labels)
            
            ftrs,outputs = model(images)
            features.append(ftrs)

            _,preds = torch.max(outputs,1)
            n_samples += labels.size(0)
            n_correct += (preds == labels).sum().item()
                
        accuracy = n_correct/float(n_samples)

        print(f'Accuracy of model on {phase} set = {(100.0 * accuracy):.4f} %')

    return features,true_labels
        

In [ ]:
features = []
true_labels = []

In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)

features,true_labels = eval_model_extract_features(features,true_labels,model,dataloader=train_loader,phase='training')

print(len(features),len(true_labels))

In [ ]:
features,true_labels = eval_model_extract_features(features,true_labels,model,dataloader=val_loader,phase='validation')

print(len(features),len(true_labels))

In [ ]:
ftrs = features.copy() 
lbls = true_labels.copy()

In [ ]:
for i in range(len(ftrs)):
    ftrs[i]=ftrs[i].cpu().numpy()

ftrs[0].shape

In [ ]:
for i in range(len(lbls)):
    lbls[i]=lbls[i].cpu().numpy()

lbls[0].shape

In [ ]:
type(ftrs),type(lbls)

In [ ]:
ftrs = np.array(ftrs)
lbls = np.array(lbls)

ftrs.shape,lbls.shape

In [ ]:
n_samples = ftrs.shape[0]*ftrs.shape[1]
n_features = ftrs.shape[2]
ftrs = ftrs.reshape(n_samples,n_features)

print(ftrs.shape)

In [ ]:
n_lbls = lbls.shape[0]
lbls = lbls.reshape(n_lbls)

print(lbls.shape)

In [ ]:
# save to csv
ftrs_df = pd.DataFrame(ftrs)
ftrs_df.to_csv('/content/gdrive/MyDrive/Kaggle/resnet50_FC_features_512.csv',index=False)

# reloading the saved csv into a df

ftrs_df = pd.read_csv('/content/gdrive/MyDrive/Kaggle/resnet50_FC_features_512.csv')
ftrs_df

In [ ]:
# appending labels to the feature set
ftrs_df['label'] = lbls

ftrs_df.head()

In [ ]:
ftrs_df.to_csv('/content/gdrive/MyDrive/Kaggle/resnet50_FC_512_features_with_labels1.csv',index=False)

print('feature set saved successfully !')

In [ ]:
# save model
MODEL_PATH = '/content/gdrive/MyDrive/Kaggle/resnet50_TL_model_94(2)%acc.pth'
torch.save(model.state_dict(),MODEL_PATH)

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model = ConvNet(model,num_classes)
model = model.to(device)
model.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
with torch.no_grad():
        n_correct = 0
        n_samples = 0
        pretrained_dict = ...
        
         
        #model.load_state_dict(torch.load(MODEL_PATH))
        
        model.eval()

        image=total_dataset[0][0]
        print(image)
        picture=image.clone().numpy()

        print(picture)

        images = image.to(device).unsqueeze(0)
        ftrs,outputs = model(images)
        #features.append(ftrs)
        _,preds = torch.max(outputs,1)
        print(preds)

        from PIL import Image
        import numpy as np
        w, h = 224, 224
        # red patch in upper left
        picture=np.moveaxis(picture,0,2)
      
        img = Image.fromarray(picture, 'RGB')

        img.save('my.png')
        img.show()
        print(picture.shape)

        '''for images in prediction_dataset:
          images = images.to(device)

          ftrs,outputs = model(images)
          features.append(ftrs)
          _,preds = torch.max(outputs,1)
          print(preds)
            #n_samples += labels.size(0)
            #n_correct += (preds == labels).sum().item()'''

In [ ]:
from PIL import Image
import numpy as np


w, h = 224, 224
data = np.zeros((h, w, 3), dtype=np.uint8)
 # red patch in upper left

img = Image.fromarray(picture, 'RGB')
img.save('my.png')
img.show()
print(picture.size())

In [ ]:
from PIL import Image
import numpy as np

h = 224
w = 224

pic = Image.open("/content/daisy.jpg")
pix = transformations['train'](pic)
print(pix.size())

with torch.no_grad():
  model.eval()
  images = pix.to(device).unsqueeze(0)
  ftrs,outputs = model(images)
  _,preds = torch.max(outputs,1)
  print(outputs)